#기본 세팅

In [1]:
pip install xlrd

In [2]:
pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 28.1 MB/s eta 0:00:00


In [3]:
pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.8/493.8 kB 7.0 MB/s eta 0:00:00


In [25]:
import pandas as pd
import requests
import re
import konlpy
import numpy as np
from konlpy.tag import Okt
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


#데이터 불러오기

원본데이터

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df_original = pd.read_csv("/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/추천서비스_최종데이터(10.26).csv",
            encoding = 'utf-8-sig',
            dtype = {'ISBN' : "str"})
df = df_original.copy()
df.loc[df['요약'] == '#NAME?', '요약'] = ' '

In [24]:
# LDA, dictionary, corpus 모델 불러오기
lda_model = LdaModel.load('/content/drive/MyDrive/도서분석 AI 용 데이터/최종 model/lda_model.model')
dictionary = Dictionary.load('/content/drive/MyDrive/도서분석 AI 용 데이터/최종 model/dictionary.dict')
corpus = corpora.MmCorpus('/content/drive/MyDrive/도서분석 AI 용 데이터/최종 model/corpus.mm')
#LDA 토픽분포 불러오기
doc_topic_distributions = np.load("/content/drive/MyDrive/도서분석 AI 용 데이터/최종 model/doc_topic_distributions.npy")
#embeddings 불러오기
embeddings = np.load("/content/drive/MyDrive/도서분석 AI 용 데이터/s_bert_embeddings.npy")

In [26]:
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#도서 검색
new_data에 [group 된 CID, lda 결과, 요약된 책 소개]

In [41]:
#검색어
query = "모루카"

In [42]:
TTBKey = "ttbleejunho09091938001"

url = f"http://www.aladin.co.kr/ttb/api/ItemSearch.aspx?ttbkey={TTBKey}&Query={query}&QueryType=Title&MaxResults=10&start=1&SearchTarget=Book&output=JS&Version=20131101"
res = requests.get(url)
res.raise_for_status()

original_description = res.json()['item'][0]['description']

#1. CID
grouped_cid = 0
lda_topic = 0

In [43]:
#2. LDA 처리
#new_description 전처리
new_description = re.sub(r'[^\w\s가-힣]', ' ', original_description)  # 특수문자 제거
new_description = re.sub(r'[\n\d]', ' ', new_description)  # 줄바꿈, 숫자 제거

#Okt, 불용어사전 준비
Okt = konlpy.tag.Okt()
stopword_data_path = '/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/stopwords-ko.txt'
with open(stopword_data_path, 'r', encoding='utf-8') as f:
  stopwords = [line.strip() for line in f.readlines()]

#Okt, 불용어제거 실행
Okt_morphs = Okt.pos(new_description)
new_description = ' '.join([word for word, pos in Okt_morphs if pos == 'Noun' and word not in stopwords and len(word)>1])

new_data = [grouped_cid, lda_topic, original_description]

#책 소개 SBERT - FAISS 분석 : result_sbert
책 소개 데이터 Sentence BERT로 임베딩 후 FAISS분석

S-BERT로 임베딩

FAISS 사용

In [44]:
# 4. FAISS 인덱스 생성
d = embeddings.shape[1]  # 벡터 차원
index = faiss.IndexFlatL2(d)  # L2 거리 기반 인덱스 생성

index.add(embeddings)  # 임베딩 데이터 추가

# 5. 새로운 데이터로 유사한 데 이터 검색
def find_similar_data(sentence, top_k=1):
  query_embedding = model.encode(sentence).reshape(1,-1) #-1은 자동으로 계산하라는 뜻.

  distances, similar_indices = index.search(query_embedding, top_k)  # 유사도 검색
  return distances, similar_indices

# 테스트
result_sbert_distances, result_sbert_indexes = find_similar_data(new_data[2], top_k=len(df))

In [45]:
#index & 유사도
result_sbert = pd.DataFrame({
    'index': result_sbert_indexes[0],
    'faiss_distance': result_sbert_distances[0]
    })
result_sbert['faiss_similarity'] = np.max(result_sbert['faiss_distance']) - result_sbert['faiss_distance']

#LDA 유사도 분석

새로들어온 데이터 lda 분석해 같은 형태로 전환

In [46]:
new_doc = new_description.split() #토큰화
new_doc2bow = dictionary.doc2bow(new_doc) #doc2bow
new_doc_lda = lda_model.get_document_topics(new_doc2bow) #lda 분석

new_doc_topic_distribution = lda_model.get_document_topics(new_doc_lda, minimum_probability=0)
new_doc_topic_distribution = [prob for _, prob in new_doc_topic_distribution]

비교

In [47]:
similarities = cosine_similarity(
    [new_doc_topic_distribution],  # 새 문서의 토픽 분포
    doc_topic_distributions        # 기존 문서들의 토픽 분포
)

# 가장 유사도가 높은 문서 찾기
most_similar_index = np.argmax(similarities)  # 유사도가 가장 높은 문서의 인덱스
most_similar_doc = doc_topic_distributions[most_similar_index]  # 해당 문서의 토픽 분포
most_similar_score = similarities[0, most_similar_index]  # 유사도 값

In [48]:
#상위 100개 index & 유사도
result_lda = pd.DataFrame({
    'index': range(len(similarities[0])),
    'lda_similarity': similarities[0]
    })

In [49]:
result_lda_sorted = result_lda.sort_values(by='lda_similarity', ascending=False)

#결과 내기

In [50]:
lda_min = np.min(result_lda['lda_similarity'])
lda_max = np.max(result_lda['lda_similarity'])
result_lda['lda_norm'] = result_lda['lda_similarity'].apply(lambda x: (x - lda_min) / (lda_max - lda_min))

sbert_min = np.min(result_sbert['faiss_similarity'])
sbert_max = np.max(result_sbert['faiss_similarity'])
result_sbert['faiss_norm'] = result_sbert['faiss_similarity'].apply(lambda x: (x - sbert_min) / (sbert_max - sbert_min))

In [51]:
a = 0.75 #Faiss 가중치
b = 0.25 #ㅣㅇㅁ rkwndcl

result_total = pd.merge(result_sbert, result_lda, on='index', how='inner')
result_total['total'] = (result_total['faiss_norm'] * a + result_total['lda_norm'] * b)*100
result_total = result_total.sort_values(by='total', ascending=False)
result_total.head(10)

,index,faiss_distance,faiss_similarity,faiss_norm,lda_similarity,lda_norm,total
83,22407,172.093414,474.255341,0.919539,0.797124,0.816919,89.388371
875,6886,204.831863,441.516907,0.856062,0.952989,0.985343,88.838207
870,39631,204.742950,441.605804,0.856234,0.946611,0.978451,88.678841
925,41273,205.869385,440.479370,0.854050,0.948102,0.980062,88.555304
1072,32433,208.164093,438.184662,0.849601,0.952257,0.984552,88.333876
137,4939,177.740082,468.608673,0.908590,0.784245,0.803003,88.219337
1128,29741,209.012299,437.336456,0.847956,0.933657,0.964453,87.708053
324,33655,189.051483,457.297272,0.886659,0.792670,0.812105,86.802026
50,2395,165.849228,480.499512,0.931646,0.660952,0.669774,86.617767
1325,5892,212.316818,434.031921,0.841549,0.900611,0.928744,86.334785


In [52]:
res = pd.DataFrame({
    'ISBN' : df.iloc[result_total.head(10)['index'].values]['ISBN'].values,
    '점수' : result_total.head(10)['total'].values
})
res

,ISBN,점수
0,9791185014470,89.388371
1,9788982733208,88.838207
2,9791168411937,88.678841
3,9791168415744,88.555304
4,9791158511180,88.333876
5,9791158884680,88.219337
6,9788901220970,87.708053
7,9791165010034,86.802026
8,9788954608640,86.617767
9,9788979192223,86.334785
